In [2]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

## Data loading and cleasing

In [128]:

employment = pd.read_csv('new_employment.csv')
income = pd.read_csv('new_income.csv')
pop =  pd.read_csv('new_population.csv')
fac = pd.read_csv('new_child_care.csv')
loc = pd.read_csv('new_potential_loc.csv')

# population is uniform within each slot
pop['10-12'] = pop['10-14'] * 3 / 5

# identifying high-demand zipcodes
zip = income.merge(employment, how="left", left_on="zip_code", right_on="zip_code")
zip['is_high_demand'] = (zip['employment rate'] >= 0.6) | (zip['average income'] < 60000)

# arranging cols to make a zip dataframe containing information on the area
slots = fac.groupby("zip_code").sum()[['infant_capacity', 'toddler_capacity', 'preschool_capacity', 'school_age_capacity', 'children_capacity', 'total_capacity']]
zip = zip.merge(slots, how="left", left_on="zip_code", right_on="zip_code")
zip = zip.merge(pop[['zip_code', '-5', '5-9', '10-12']], left_on="zip_code", right_on="zip_code")

#identifying deserts
zip['is_desert'] = (zip['is_high_demand'] & (zip['total_capacity'] <= 1/2 * (zip['-5'] + zip['5-9'] + zip['10-12']))) | \
                (~zip['is_high_demand'] & (zip['total_capacity'] <= 1/3 * (zip['-5'] + zip['5-9'] + zip['10-12'])))


#print(pop)
fac = fac.merge(pop,how="left",left_on="zip_code", right_on="zip_code")
#test = fac.groupby('facility_id').count()
#print(test[test["zip_code"]>1])



# cleaning data
fac = fac[fac['total_capacity'] > 0]
zip['demand_child'] = zip['-5'] + zip['5-9'] + zip['10-12']
zip['under5_capacity'] = zip['infant_capacity'] + zip['toddler_capacity'] + zip['preschool_capacity']
fac['under5_capacity'] = fac['infant_capacity'] + fac['toddler_capacity'] + fac['preschool_capacity']
zip = zip[zip['demand_child'] > 0]


# declair variables useful throughout the project
new_fac_cost = {
    'size': ['small', 'medium', 'large'],
    'total_slots': [100, 200, 400],
    'under5_slots': [50, 100, 200],
    'cost': [65000, 95000, 115000]
}
new_fac_cost = pd.DataFrame(new_fac_cost)
new_fac_cost


fac = fac.reset_index(drop=True)
zip = zip.reset_index(drop=True)

zip.head(20)
fac.head(10)



# identifying key elements
zipcode = zip['zip_code'].tolist()
new_fac_size = ['small', 'medium', 'large']
exist_fac = fac['facility_id'].unique()
age = ['under5', 'over5']


## Solving for question 1: Child Solution

### 1.1 setting variables

In [129]:
load_data()
exist_fac = fac[fac['total_capacity'] <= 500]['facility_id'].unique()

m = Model("child_solution")

# Decision Variables：new facilities & slot expansion
x = {}
for i in zipcode:
    for j in new_fac_size:
        x[i, j] = m.addVar(vtype = GRB.INTEGER, name = f"x_{i}_{j}")

y = {}
for f in exist_fac:
    for a in age:
        y[f, a] = m.addVar(vtype = GRB.INTEGER, name = f"y_{f}_{a}")

### 1.2 Target Function

In [130]:
# Objective Function：Min the cost of...
new_fac_cost_expr = quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['cost'].values[0] for i in zipcode for j in new_fac_size)
expand_cost_expr = sum(((20000 + (fac[fac['facility_id'] == f]['total_capacity'].values[0]) * 200) * (y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] + 100 * y[f, 'under5']) for f in exist_fac)

m.setObjective(new_fac_cost_expr + expand_cost_expr, GRB.MINIMIZE)

### 1.3 Adding constraints

In [131]:
# Constraints:
# (1) no desert in both high demand and normal demand region
for i in zipcode:
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == i]['total_capacity'].values[0]) / zip[zip['zip_code'] == i]['demand_child'].values[0] >=
        1/2 * zip[zip['zip_code'] == i]['is_high_demand'].values[0], f"no_desert_high_demand_{i}")
    
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == i]['total_capacity'].values[0]) / zip[zip['zip_code'] == i]['demand_child'].values[0] >=
        1/3, f"no_desert_low_demand_{i}")

# (2) under5 take up a higher coverage percantage
for i in zipcode:
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['under5_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, 'under5'] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist()) +
         zip[zip['zip_code'] == i]['under5_capacity'].values[0]) >=
        2/3 * zip[zip['zip_code'] == i]['-5'].values[0], f"under5_demand_{i}")
    
# (3) expansion maximum scale
for f in exist_fac:
    m.addConstr((y[f, 'under5'] + y[f, 'over5']) <= 0.20 * fac[fac['facility_id'] == f]['total_capacity'].values[0], f"expansion_restriction1_{f}")
    m.addConstr(y[f, 'under5'] + y[f, 'over5'] + fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 500 , f"expansion_restriction2_{f}")

### 1.4 solve and save variable results

In [132]:
# Solve
m.optimize()

x_result_list = []
y_result_list = []


if m.status == GRB.OPTIMAL:
    print("Optimal solution found. Results:")
    
    for i in zipcode:
        for j in new_fac_size:
            zip_list = []
            zip_list.append(i)
            zip_list.append(j)
            zip_list.append(x[i, j].x)

            x_result_list.append(zip_list)
                #print(f"Region {i}: Build {x[i, j].x} {j} size new facilities.")

    q1_x_result_copy = pd.DataFrame(x_result_list ,columns=["zipcode","size","amount_built"])
    print(q1_x_result_copy)
    q1_x_result_copy.to_csv('q1_x_result.csv')
                
    
    new_facilities_df = pd.DataFrame(new_facilities_data)
    
    for f in exist_fac:
        
        for a in age:
            fac_list = []
            fac_list.append(f)
            fac_list.append(a)
            fac_list.append(y[f, a].x)
            y_result_list.append(fac_list)
    q1_y_result_copy = pd.DataFrame(y_result_list ,columns=["facility_code","category","slot_expanded"])
    q1_y_result_copy.to_csv('q1_y_result.csv')


elif m.status == GRB.INFEASIBLE:
    print("The model is infeasible. Check constraints or data.")
elif m.status == GRB.UNBOUNDED:
    print("The model is unbounded. Consider revising the model.")
else:
    print(f"Optimization was stopped with status {m.status}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 32542 rows, 32542 columns and 141834 nonzeros
Model fingerprint: 0x901fe872
Variable types: 0 continuous, 32542 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-05, 2e+02]
  Objective range  [2e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 8e+03]
Found heuristic solution: objective 3.855950e+08
Presolve removed 32503 rows and 32465 columns
Presolve time: 0.34s
Presolved: 39 rows, 77 columns, 191 nonzeros
Found heuristic solution: objective 3.400448e+08
Variable types: 0 continuous, 77 integer (28 binary)

Root relaxation: interrupted, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

## Solving for question 2: Realistic Capacity Expansion and Distance

In [133]:

employment = pd.read_csv('new_employment.csv')
income = pd.read_csv('new_income.csv')
pop =  pd.read_csv('new_population.csv')
fac = pd.read_csv('new_child_care.csv')
loc = pd.read_csv('new_potential_loc.csv')

# population is uniform within each slot
pop['10-12'] = pop['10-14'] * 3 / 5

# identifying high-demand zipcodes
zip = income.merge(employment, how="left", left_on="zip_code", right_on="zip_code")
zip['is_high_demand'] = (zip['employment rate'] >= 0.6) | (zip['average income'] < 60000)

# arranging cols to make a zip dataframe containing information on the area
slots = fac.groupby("zip_code").sum()[['infant_capacity', 'toddler_capacity', 'preschool_capacity', 'school_age_capacity', 'children_capacity', 'total_capacity']]
zip = zip.merge(slots, how="left", left_on="zip_code", right_on="zip_code")
zip = zip.merge(pop[['zip_code', '-5', '5-9', '10-12']], left_on="zip_code", right_on="zip_code")

#identifying deserts
zip['is_desert'] = (zip['is_high_demand'] & (zip['total_capacity'] <= 1/2 * (zip['-5'] + zip['5-9'] + zip['10-12']))) | \
                (~zip['is_high_demand'] & (zip['total_capacity'] <= 1/3 * (zip['-5'] + zip['5-9'] + zip['10-12'])))


#print(pop)
fac = fac.merge(pop,how="left",left_on="zip_code", right_on="zip_code")
#test = fac.groupby('facility_id').count()
#print(test[test["zip_code"]>1])



# cleaning data
fac = fac[fac['total_capacity'] > 0]
zip['demand_child'] = zip['-5'] + zip['5-9'] + zip['10-12']
zip['under5_capacity'] = zip['infant_capacity'] + zip['toddler_capacity'] + zip['preschool_capacity']
fac['under5_capacity'] = fac['infant_capacity'] + fac['toddler_capacity'] + fac['preschool_capacity']
zip = zip[zip['demand_child'] > 0]


# declair variables useful throughout the project
new_fac_cost = {
    'size': ['small', 'medium', 'large'],
    'total_slots': [100, 200, 400],
    'under5_slots': [50, 100, 200],
    'cost': [65000, 95000, 115000]
}
new_fac_cost = pd.DataFrame(new_fac_cost)
new_fac_cost


fac = fac.reset_index(drop=True)
zip = zip.reset_index(drop=True)

zip.head(20)
fac.head(10)



# identifying key elements
zipcode = zip['zip_code'].tolist()
new_fac_size = ['small', 'medium', 'large']
exist_fac = fac['facility_id'].unique()
age = ['under5', 'over5']


# defining usful functions in question 2
from haversine import haversine, Unit

potential_loc = pd.read_csv("new_potential_loc.csv")
potential_loc['location_string'] = potential_loc['latitude'].astype(str)+","+potential_loc['longitude'].astype(str)
potential_loc.head(10)

def loc_distance(lat1, lon1, lat2,lon2):
    distance = haversine((lat1, lon1), (lat2, lon2), unit=Unit.MILES)
    return distance
#print(cost_func(300,13))
#print(calc_dist([70,40],[68,38]))


### 2.1 setting variables

In [134]:

import gurobipy as gp
from gurobipy import GRB
m = Model("Realistic Capacity Expansion and Distance")

#load_data()
#exist_fac = fac[fac['total_capacity'] <= 500]['facility_id'].unique()
x = {}
loc_list = []
loc_str_list = []

for row in range(len(potential_loc)):
    #print()
    loc = potential_loc.loc[row,['latitude','longitude']].tolist()
    loc_list.append(loc)
    lat = loc[0]
    lon = loc[1]
    loc_string = str(lat)+","+str(lon)
    loc_str_list.append(loc_string)
    #print(loc)
    for size in new_fac_size:
        x[loc_string, size] = m.addVar(vtype = GRB.BINARY, name = f"x_{loc_string}_{size}")
        #print(loc_string)
        #print(zipzip,loc,size)



'''
# The following snippet only needs to run once to produce closer than 0.06 pairs to csv

# find all pairs of locations with distance <= 0.06 and constrain on their spontaneous establishments later
pairs = []
for code in potential_loc['zip_code'].unique():
    print(code)
    workbook = potential_loc[potential_loc['zip_code']==code]
    for i in workbook['location_string'].unique().tolist():
        for j in workbook['location_string'].unique().tolist():
            if loc_str_list.index(i) < loc_str_list.index(j):
                #print(i,j)
                if loc_distance(float(i.split(",")[0]),float(i.split(",")[1]),float(j.split(",")[0]),float(j.split(",")[1])) <= 0.06:
                    #print(f"found {i}, {j}")
                    pairs.append((i,j))
                    #pairs.append((code,i,j))

pairs_copy = pd.DataFrame(pairs,columns=['l1','l2'])
pairs_copy.to_csv('pairs_save.csv')
'''

'''
# find all locations with distance <= 0.06 to established facilities and constrain on their spontaneous establishments later
near_establishment_locations = []
for code in potential_loc['zip_code'].unique():
    #print(code)
    workbook = potential_loc[potential_loc['zip_code']==code]
    work_fac = fac[fac['zip_code']==code].reset_index(drop = True)
    for i in workbook['location_string'].unique().tolist():
        for row in range(len(work_fac)):
            lat1 = float(i.split(",")[0])
            lon1 = float(i.split(",")[1])
            lat2 = float(work_fac.loc[row,'latitude'])
            lon2 = float(work_fac.loc[row,'longitude'])
            if loc_distance(lat1,lon1,lat2,lon2) <=0.06:
                near_establishment_locations.append(i)
near_establishment_locations_copy = pd.DataFrame(near_establishment_locations,columns=['location'])
near_establishment_locations_copy.to_csv('near_establishment_locations_save.csv')
'''

# number of expansion slots for 0-5y to existing factories 
y = {}
for f in exist_fac:
    y[f] = m.addVar(vtype = GRB.INTEGER, name = f'y_{f}')

# number of expansion slots to existing factories for each step, 
steps = ['step1', 'step2', 'step3']
z = {}
for f in exist_fac:
    for s in steps:
        z[f, s] = m.addVar(vtype = GRB.INTEGER, name = f'step_{f}_{s}')

#load saved <0.06miles pairs
pairs_df = pd.read_csv("pairs_save.csv")
pairs = []
for i in range(len(pairs_df)):
    pairs.append((pairs_df.loc[i,"l1"],pairs_df.loc[i,"l2"]))


#load saved <0.06miles pairs
near_establishment_locations_df = pd.read_csv("near_establishment_locations_save.csv")
near_establishment_locations = []
for i in range(len(near_establishment_locations_df)):
    near_establishment_locations.append((near_establishment_locations_df.loc[i,"location"]))
        

### 2.2 target function

In [135]:
expand_cost_expr = quicksum((20000+200*fac[fac['facility_id'] == f]['total_capacity'].values[0])*z[f, 'step1']/fac[fac['facility_id'] == f]['total_capacity'].values[0]
                       +(20000+400*fac[fac['facility_id'] == f]['total_capacity'].values[0])*z[f, 'step2']/fac[fac['facility_id'] == f]['total_capacity'].values[0]
                       +(20000+1000*fac[fac['facility_id'] == f]['total_capacity'].values[0])*z[f, 'step3']/fac[fac['facility_id'] == f]['total_capacity'].values[0] for f in exist_fac)
    
new_fac_cost_expr = quicksum(x[j, k] * new_fac_cost[new_fac_cost['size'] == k]['cost'].values[0] for j in loc_str_list for k in new_fac_size)

young_slots_expr = gp.quicksum(100 * y[f] for f in exist_fac)

m.setObjective(new_fac_cost_expr + expand_cost_expr + young_slots_expr, GRB.MINIMIZE)

### 2.3 adding constrains

In [136]:
from gurobipy import quicksum, min_, Var, LinExpr


# (0) constraining on the dist<=0.06 pairs
print(len(pairs))
for (l1,l2) in pairs:
    m.addConstr(
        quicksum(x[l1,size]+x[l2,size] for size in new_fac_size) <= 1, name = f"distance_restriction_{l1},{l2}"
    )
print('inter-new-establishments distance restriction done')

# (0) constraining on the dist to existing facilities (less than 0.06 miles
print(len(near_establishment_locations))
for loc in near_establishment_locations:
    m.addConstr(
        quicksum(x[loc,size] for size in new_fac_size) == 0, name = f"established_distance_restriction_{loc}"
    )
print('near_establishment_locations constraint done')


#(1) no desert
for code in zipcode:
    i_ = potential_loc[potential_loc["zip_code"]==code]["location_string"].to_list()
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(z[f,s] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist() for s in steps) +
         zip[zip['zip_code'] == code]['total_capacity'].values[0]) / zip[zip['zip_code'] == code]['demand_child'].values[0] >=
        1/2 * zip[zip['zip_code'] == code]['is_high_demand'].values[0], f"no_desert_high_demand_{code}")
    
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(z[f,s] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist() for s in steps) +
         zip[zip['zip_code'] == code]['total_capacity'].values[0]) / zip[zip['zip_code'] == code]['demand_child'].values[0] >=
        1/3, f"no_desert_low_demand_{code}")

print('no desert constraint done')

#(2) under5
for code in zipcode:
    i_ = potential_loc[potential_loc["zip_code"]==code]["location_string"].to_list()
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['under5_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(y[f] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist()) +
         zip[zip['zip_code'] == code]['under5_capacity'].values[0]) >=
        2/3 * zip[zip['zip_code'] == code]['-5'].values[0], f"under5_demand_{code}")

print('under-5 requirement done')

#(3) expansion maximum
for f in exist_fac:
    m.addConstr(z[f, 'step1'] / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.1)
    m.addConstr(z[f, 'step2'] / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.05)
    m.addConstr(z[f, 'step3'] / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.05)
    if fac[fac['facility_id'] == f]['total_capacity'].values[0] < 500:
        m.addConstr(quicksum(z[f, s] for s in steps) <= 500-fac[fac['facility_id'] == f]['total_capacity'].values[0])
    else:
        m.addConstr(quicksum(z[f, s] for s in steps) == 0)
    m.addConstr(y[f] <= quicksum(z[f, s] for s in steps))
    m.addConstr(y[f] >= 0)
    for s in steps:
        m.addConstr(z[f, s] >= 0)

for i in potential_loc["location_string"].to_list():
    m.addConstr(quicksum(x[i, j] for j in new_fac_size) <= 1, f'new_facility_num_{i}')

print('expansion constrains done')


m.update()




38632
inter-new-establishments distance restriction done
6333
near_establishment_locations constraint done
no desert constraint done
under-5 requirement done
expansion constrains done


In [137]:
m.optimize()

# Print the solution
print('Total cost: $%g' % m.objVal)
if m.status == gp.GRB.OPTIMAL:

    # save x results
    x_result_list = []
    for loc in loc_str_list:
            
            for size in new_fac_size:
                loc_results = []
                loc_results.append(loc)
                loc_results.append(size)
                loc_results.append(x[loc,size].x)
            #print(loc_results)
                x_result_list.append(loc_results)
        

    q2_x_result_copy = pd.DataFrame(x_result_list ,columns=['location','size','if_Built'])
    q2_x_result_copy.to_csv('q2_x_result.csv')

    # save y[f] results
    y_result_list = []
    for f in exist_fac:
        y_result_list.append(y[f].x)
    q2_y_result_copy = pd.DataFrame(y_result_list ,columns=["under5_expansion_slots"])
    q2_y_result_copy.set_index(exist_fac, inplace=True)
    #print(q2_y_result_copy)
    q2_y_result_copy.to_csv('q2_y_result.csv')
         
    # save z[f,s] results
    z_result_list = []
    for f in exist_fac:
        
        for s in steps:
            fac_result = []
            fac_result.append(f)
            fac_result.append(s)
            fac_result.append(z[f,s].x)
            z_result_list.append(fac_result)

    q2_z_result_copy = pd.DataFrame(z_result_list ,columns=['facility_code','step','slot_expanded'])
    q2_z_result_copy.to_csv('q2_z_result.csv')
    

else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 283120 rows, 365920 columns and 1785525 nonzeros
Model fingerprint: 0x0a93ab78
Variable types: 0 continuous, 365920 integer (306900 binary)
Coefficient statistics:
  Matrix range     [4e-05, 2e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 8e+03]
Presolve removed 282842 rows and 365289 columns (presolve time = 8s) ...
Presolve removed 282839 rows and 365287 columns
Presolve time: 8.36s
Presolved: 281 rows, 633 columns, 1371 nonzeros
Variable types: 0 continuous, 633 integer (553 binary)
Found heuristic solution: objective 3.457724e+08

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4352743e+08   2.514592e+02   0.000000e+00      8s
     248    3.4556380e+08   0.000000e+00   0